# Step 1 - Prepare network data
## Project: Growing Urban Bicycle Networks

This notebook downloads bicycle and street network data sets and prepares them for analysis.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-06-09  
Last modified: 2021-03-10

## Preliminaries

### Parameters

In [1]:
debug = True # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

Loaded parameters.



### Setup

In [2]:
%run -i path.py
#%run -i setup.py
%run -i setupCPH.py

%load_ext watermark
%watermark -n -v -m -g -iv
import statistics

Loaded PATH.

Setup finished.

Python implementation: CPython
Python version       : 3.8.2
IPython version      : 8.5.0

Compiler    : Clang 9.0.1 
OS          : Darwin
Release     : 18.7.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

Git hash: b795eaefba9eb6349c319ca07997ebd31e033f57

osmnx     : 0.16.2
networkx  : 2.8.6
pyproj    : 3.4.0
fiona     : 1.8.21
geopandas : 0.11.1
geojson   : 2.5.0
osgeo     : 3.2.1
csv       : 1.0
watermark : 2.3.1
shapely   : 1.8.4
numpy     : 1.23.3
igraph    : 0.9.1
matplotlib: 3.6.0
sys       : 3.8.2 | packaged by conda-forge | (default, Apr 24 2020, 07:56:27) 
[Clang 9.0.1 ]
pandas    : 1.4.4



### Functions

In [3]:
%run -i functions.py

Loaded functions.



## Load carall graph

In [4]:
G_carall = ox.load_graphml('../../bikenwgrowth_external/data/copenhagen/bikedata/G_carall_graphml.graphml')

In [ ]:
btw_length = nx.edge_betweenness_centrality(G_carall,  weight='length')

In [ ]:
btw_bcount_attr = nx.edge_betweenness_centrality(G_carall,  weight='bcount_attr')

In [ ]:
btw_pop_den_attr = nx.edge_betweenness_centrality(G_carall,  weight='pop_den_attr')

In [ ]:
with open('../../bikenwgrowth_external/data/copenhagen/bikedata/btw_length.pickle', 'wb') as y:
    pickle.dump(btw_length, y)

In [ ]:
with open('../../bikenwgrowth_external/data/copenhagen/bikedata/btw_bcount_attr.pickle', 'wb') as y:
    pickle.dump(btw_bcount_attr, y)

In [ ]:
with open('../../bikenwgrowth_external/data/copenhagen/bikedata/btw_pop_den_attr.pickle', 'wb') as y:
    pickle.dump(btw_pop_den_attr, y)

In [ ]:
# btw_dict, og for btw in btw_dict normaliser!

In [ ]:
btwlist = [btw_length,btw_bcount_attr,btw_pop_den_attr,btw_bcount_pop_den_attr]
for betweenness in btwlist:
    max_x = max(betweenness.values())
    min_x = min(betweenness.values())

# normalize each value in the dict
    for k in betweenness:
        betweenness[k] = fn(betweenness[k], max_x, min_x)

In [ ]:
#normalisation

fn = lambda value, x_max, x_min: (value - x_min) / (x_max - x_min)
btw3 = betweenness
# In each inner dict

# find min and max values
max_x = max(betweenness.values())
min_x = min(betweenness.values())

# normalize each value in the dict
for k in betweenness:
    btw3[k] = fn(betweenness[k], max_x, min_x)

In [ ]:
#This plot is edge betweenness centrality based on the 'bc_len_attr' attribute 
lanes = ox.graph_to_gdfs(G_carall, nodes=False)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


ac = [btw3[k] for k in btw3]
#ac = [0.08 if (edges_dict[k]) == meancountfraction else 0.8 for k in edges_dict]
lc = [btw3[k]*20 for k in btw3]

ax = world[world.name == "Copenhagen municipality"].plot(color='black', edgecolor='black', figsize=(50,50),bgcolor = 'black')
#bikelanes.plot(ax=ax, color='red')
lanes.plot(ax=ax, color='blue', alpha= ac, linewidth = lc)


plt.show()